In [2]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

import joblib

import re

In [12]:
file_path = "병합_청약매물_목록_정보.csv"

df = pd.read_csv(file_path, encoding='cp949')

### 불필요 컬럼 제거

In [36]:
def filter_unnecessary_columns(df):

    # 민간주택만 필터링 (공공주택은 가점제 없음)
    df = df[df['주택상세구분코드명'] != '국민']
    
    # 불필요 칼럼 확인
    unnecessary_columns = []
    
    # Case 1) 칼럼 값이 하나라면 불필요 칼럼
    for column in df.columns:
        if len(df[column].unique()) == 1:
            unnecessary_columns.append(column)
    
    # Case 2) 홈페이지 주소, 문의처 칼럼
    for column in df.columns:
        if column in ['홈페이지주소', '모집공고홈페이지주소', '문의처']:
            unnecessary_columns.append(column)
    
    # Case 3) 특별공급 관련 칼럼
    for column in df.columns:
        if '특별공급' in column:
            unnecessary_columns.append(column)
    
    # Case 4) 청약접수종료일을 제외한 나머지 일자 칼럼
    for column in df.columns:
        if column != '청약접수종료일':
            if ('시작일' in column or '종료일' in column):
                unnecessary_columns.append(column)
    
    # Case 5) 그 외 기타 칼럼
    unnecessary_columns.extend(["주택관리번호", "모델번호", '주택관리번호', '공급위치우편번호', '모집공고일', '당첨자발표일', '입주예정월'])
    
    # 시공사랑 시행사는 불필요할것같아서 추가로 제거할게요
    unnecessary_columns.extend(['건설업체명_시공사', '사업주체명_시행사'])
    
    # 중복된 칼럼 제거
    unnecessary_columns = list(set(unnecessary_columns))
    
    # 불필요한 칼럼 삭제
    df = df.drop(columns=unnecessary_columns)
    
    return df


In [37]:
# 사용 예시
# df = filter_unnecessary_columns(df)


### 전처리

In [38]:
def split_housing_type(df):
    if '주택형' in df.columns:
        # Extract the '전용면적' (before the '.') and remove leading zeros
        df['전용면적'] = df['주택형'].apply(lambda x: int(x.split('.')[0].lstrip('0')))
        
        # Extract the '평면유형' (last character)
        df['평면유형'] = df['주택형'].apply(lambda x: x[-1])
        
        # Drop the original '주택형' column
        df = df.drop(columns=['주택형'])
    
    return df

In [39]:
# 예시
# df = split_housing_type(df)

경쟁률 미달 처리

In [40]:
def preprocessing_applicant_rate(df):

    def process_rate(row):
        # 미달인 경우 경쟁률 처리
        if '△' in str(row['경쟁률']):
            pattern = '[^0-9]'
            shortage = int(re.sub(pattern, '', str(row['경쟁률'])))
            
            if row['접수건수'] == 0:
                rate = 0
            else:
                rate = round((row['접수건수'] - shortage) / row['접수건수'], 2)
        # 쉼표 제거 후 float으로 변환
        else:
            rate = float(str(row['경쟁률']).replace(',', ''))
        
        # 미달여부 판단
        shortage_status = 'Y' if rate < 1 else 'N'
        
        return pd.Series({'경쟁률': rate, '미달여부': shortage_status})

    df[['경쟁률', '미달여부']] = df.apply(process_rate, axis=1)
    
    return df


In [41]:
# 사용예시
# df = preprocessing_applicant_rate(df)

경쟁률 NaN값 처리

In [42]:
def fill_nan_with_zero(df):
    df["경쟁률"] = df["경쟁률"].fillna(0)
    return df

In [43]:
# 예시
# df = fill_nan_with_zero(df)

공급금액 데이터 병합

In [ ]:
def add_estate_price(df):
    # 공급금액 데이터 불러오기
    df_estate_price = pd.read_csv("청약매물_공급금액 (서울, 경기, 인천).csv", encoding='cp949')
    df_estate_price = df_estate_price[['공고번호', '주택형', '공급금액(최고가 기준)']]
    
    print(df_estate_price)

    # 원본 데이터에 공급금액 칼럼 추가하기
    df = pd.merge(df, df_estate_price, on=['공고번호', '주택형'], how='inner')

    return df

             공고번호        주택형  공급금액(최고가 기준)
0      2024000706  075.9493A   664170000.0
1      2024000706  075.9493A   664170000.0
2      2024000706  075.9493A   664170000.0
3      2024000706  075.9493A   664170000.0
4      2024000706  075.9347B   591150000.0
...           ...        ...           ...
19625  2020000009   074.9100   456000000.0
19626  2020000009   074.8900   456000000.0
19627  2020000009   074.8900   456000000.0
19628  2020000009   074.8900   456000000.0
19629  2020000009   074.8900   456000000.0

[19630 rows x 3 columns]
0


### 데이터 파이프 라인 준비

1. 컬럼 전처리

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler

# FunctionTransformer로 함수 변환
filter_transformer = FunctionTransformer(filter_unnecessary_columns)
split_transformer = FunctionTransformer(split_housing_type)
rate_transformer = FunctionTransformer(preprocessing_applicant_rate)
nan_transformer = FunctionTransformer(fill_nan_with_zero)

# 파이프라인 구성
preprocessing_pipeline = Pipeline([
    ("filter", filter_transformer),
    ("split", split_transformer),
    ("rate", rate_transformer),
    ("nan", nan_transformer)
])


In [45]:
df = preprocessing_pipeline.transform(df)


In [ ]:
df.isnull().sum()

공고번호                0
주택명                 0
공급지역코드              0
공급지역명               0
공급위치                0
공급규모                0
청약접수종료일             0
투기과열지구              0
조정대상지역              0
분양가상한제              0
정비사업                0
공공주택지구              0
대규모택지개발지구           0
수도권내민영공공주택지구        0
신청자 30대 이하          0
신청자 40대             0
신청자 50대             0
신청자 60대 이상          0
당첨자 30대 이하          0
당첨자 40대             0
당첨자 50대             0
당첨자 60대 이상          0
공급세대수               0
순위                  0
거주코드                0
거주지역                0
접수건수                0
경쟁률                 0
최저당첨가점          25902
평균당첨가점          25902
최고당첨가점          25902
전용면적                0
평면유형                0
미달여부                0
dtype: int64

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36848 entries, 0 to 38317
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공고번호          36848 non-null  int64  
 1   주택명           36848 non-null  object 
 2   공급지역코드        36848 non-null  int64  
 3   공급지역명         36848 non-null  object 
 4   공급위치          36848 non-null  object 
 5   공급규모          36848 non-null  int64  
 6   청약접수종료일       36848 non-null  object 
 7   투기과열지구        36848 non-null  object 
 8   조정대상지역        36848 non-null  object 
 9   분양가상한제        36848 non-null  object 
 10  정비사업          36848 non-null  object 
 11  공공주택지구        36848 non-null  object 
 12  대규모택지개발지구     36848 non-null  object 
 13  수도권내민영공공주택지구  36848 non-null  object 
 14  신청자 30대 이하    36848 non-null  int64  
 15  신청자 40대       36848 non-null  int64  
 16  신청자 50대       36848 non-null  int64  
 17  신청자 60대 이상    36848 non-null  int64  
 18  당첨자 30대 이하    36848 non-null  i

In [50]:
df.to_csv('train-0311.csv', index=False, encoding='cp949')